# Using statistical test let's see which stocks are the most mean reverting spread

## Building the data

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from itertools import combinations
import glob

In [15]:
list_industries = glob.glob("../data_day/*")

df_dict_stock_data = {path.split('/')[-1]:{} for path in list_industries}

for path in  list_industries:

    industry_name = path.split('/')[-1]
    list_stocks = glob.glob(path+'/*')

    for stock in list_stocks:

        stock_name = stock.split('/')[-1]
        stock_path = glob.glob(stock+'/*')[0]
        df = pd.read_csv(stock_path)
        df_dict_stock_data[industry_name][stock_name] = df

## Version for hand picked two stocks

In [99]:
df_dict_stock_data['airlines']['AAL']

,time,open,high,low,close,volume,volume_weighted
0,2023-03-08 05:00:00,16.410,16.6000,16.2701,16.59,11399587.0,16.4675
1,2023-03-09 05:00:00,16.550,16.8000,15.8450,15.88,18504951.0,16.2679
2,2023-03-10 05:00:00,15.980,16.0000,15.1100,15.46,27280184.0,15.5431
3,2023-03-13 04:00:00,15.020,15.2600,14.6900,14.85,32541745.0,14.9565
4,2023-03-14 04:00:00,15.050,15.3200,14.4850,14.66,35258905.0,14.8810
...,...,...,...,...,...,...,...
496,2025-02-28 05:00:00,14.565,14.7499,14.2700,14.35,49194960.0,14.4358
497,2025-03-03 05:00:00,14.360,14.5550,13.7700,13.87,55165105.0,14.1395
498,2025-03-04 05:00:00,13.520,13.6500,13.0150,13.35,70894421.0,13.2985
499,2025-03-05 05:00:00,13.510,14.2300,13.4900,14.09,56097559.0,13.9860


In [100]:
label_1 = 'AAL'
label_2 = 'ALGT'

df_1 = df_dict_stock_data['airlines'][label_1]
df_2 = df_dict_stock_data['airlines'][label_2]

df = pd.merge(left=df_1, right=df_2, on='time', suffixes=['_' + label_1, '_' + label_2])

df.head()

,time,open_AAL,high_AAL,low_AAL,close_AAL,volume_AAL,volume_weighted_AAL,open_ALGT,high_ALGT,low_ALGT,close_ALGT,volume_ALGT,volume_weighted_ALGT
0,2023-03-08 05:00:00,16.41,16.60,16.2701,16.59,11399587.0,16.4675,103.60,104.12,101.410,104.02,162717.0,103.1249
1,2023-03-09 05:00:00,16.55,16.80,15.8450,15.88,18504951.0,16.2679,103.95,104.10,99.355,100.77,207066.0,101.2407
2,2023-03-10 05:00:00,15.98,16.00,15.1100,15.46,27280184.0,15.5431,98.15,98.88,93.820,96.06,322162.0,95.7110
3,2023-03-13 04:00:00,15.02,15.26,14.6900,14.85,32541745.0,14.9565,91.42,93.65,89.510,91.05,275989.0,91.9271
4,2023-03-14 04:00:00,15.05,15.32,14.4850,14.66,35258905.0,14.8810,94.14,94.99,91.000,92.32,174772.0,92.7698


In [101]:
x = df['close_' + label_1]
y = df['close_' + label_2]

x = x.values.reshape(-1, 1)
x_with_cst = sm.add_constant(x)


In [102]:

model = sm.OLS(y, x_with_cst).fit()
beta = model.params.iloc[1]

print(f"The coefficients are :{model.params.iloc[1]}")

The coefficients are :8.321241151235016


In [103]:
model.params

const   -38.351472
x1        8.321241
dtype: float64

In [104]:
spread = df['close_' + label_2] - beta * df['close_' + label_1]

In [105]:
result = adfuller(spread)

In [106]:
result[1]

0.1490407547355771

## Now let's make the loop for one each industry and let's create a heatmap

### Chat gpt functions

In [ ]:
def calculate_hedge_ratio(y, x):
    """ Calculate hedge ratio using OLS regression """
    x = sm.add_constant(x)  # Add intercept
    model = sm.OLS(y, x).fit()
    return model.params[1]  # Return the slope (hedge ratio)

def perform_adf_test(spread):
    """ Perform the Augmented Dickey-Fuller test on the spread """
    result = adfuller(spread)
    return result[1]  # Return the p-value

def create_adf_heatmap(price_data):
    """ Create a heatmap of ADF test p-values for all stock pairs """
    stocks = price_data.columns
    p_values = pd.DataFrame(np.ones((len(stocks), len(stocks))), index=stocks, columns=stocks)
    
    for stock1, stock2 in combinations(stocks, 2):
        y = price_data[stock1]
        x = price_data[stock2]
        hedge_ratio = calculate_hedge_ratio(y, x)
        spread = y - hedge_ratio * x
        p_value = perform_adf_test(spread)
        
        p_values.loc[stock1, stock2] = p_value
        # p_values.loc[stock2, stock1] = p_value  # Symmetric matrix

    plt.figure(figsize=(10, 8))
    sns.heatmap(p_values, annot=True, fmt=".3f", cmap="coolwarm", linewidths=0.5)
    plt.title("ADF Test P-values Heatmap (Mean Reversion)")
    plt.show()

    return p_values

In [26]:
# # Code generate by Chat GPT to finish everything
# # Load your OHLCV data (example: only closing prices)
# price_data = pd.read_csv("your_data.csv", index_col=0, parse_dates=True)
# close_prices = price_data.pivot(columns="symbol", values="close")

# # Generate the heatmap of ADF p-values
# p_value_matrix = create_adf_heatmap(close_prices)


### Arthur's version

In [122]:
industry_list = list(df_dict_stock_data.keys())
count = 0
treshold = 0.01
pairs = []
pairs_dict = {industry: [] for industry in df_dict_stock_data.keys()}

for chosen_industry in industry_list:

    dict_industry = df_dict_stock_data[chosen_industry]
    list_stocks_in_this_industry = list(dict_industry.keys())

    list_stocks_in_this_industry

    for label_1, label_2 in combinations(list_stocks_in_this_industry, 2):
        
        df_1 = dict_industry[label_1]
        df_2 = dict_industry[label_2]

        df = pd.merge(left=df_1, right=df_2, on='time', suffixes=['_' + label_1, '_' + label_2])

        x = df['close_' + label_1]
        y = df['close_' + label_2]

        x_with_cst = sm.add_constant(x)

        model = sm.OLS(y, x_with_cst).fit()
        beta = model.params.iloc[1]

        spread = df['close_' + label_2] - beta * df['close_' + label_1]
        result = adfuller(spread)

        p_value = result[1]

        if p_value < treshold:
            print(f"The pair {label_1}/{label_2} has a p-value of {p_value}")
            count += 1
            pairs.append((label_1, label_2))
            pairs_dict[chosen_industry].append((label_1, label_2))

    #for stock in dict_industry.keys():


The pair WMT/PG has a p-value of 0.009037869962070254
The pair DEO/EL has a p-value of 0.0008374123876033327
The pair BJ/PG has a p-value of 0.001955218346254495
The pair COST/PG has a p-value of 0.0010865649657410107
The pair SAP/AVGO has a p-value of 0.0024294844463222404
The pair MU/QCOM has a p-value of 0.0018920456356948784
The pair QCOM/MSFT has a p-value of 0.009991317073002567
The pair TSM/GOOGL has a p-value of 0.005056067573448308
The pair GOOGL/AVGO has a p-value of 0.002464197912855622
The pair META/AVGO has a p-value of 0.007096385012119753
The pair KMI/BP has a p-value of 0.0006298740433253296
The pair KMI/CVX has a p-value of 0.008538719761990159
The pair KMI/DVN has a p-value of 0.0024928657838661516
The pair BP/ENB has a p-value of 0.005829841605490583
The pair BP/TRP has a p-value of 0.00012569346151618482
The pair EPD/DVN has a p-value of 0.009623001261835197
The pair HAL/ENB has a p-value of 0.0004127833849581462
The pair HAL/TRP has a p-value of 0.00078677951411216

In [124]:
print(f"The number of pairs is {len(pairs)}")

The number of pairs is 37


In [129]:
import json

with open(f'./pair_dict_treshold_{treshold}.json', 'w') as fp:
    json.dump(pairs_dict, fp)